In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [2]:
path = '../data/testA/sep_data/2019-02-16expLog.csv'

In [3]:
names = ['request_id','request_time','location_id','user_id','ad_id','ad_size',\
         'ad_bid','ad_pctr','ad_quality_ecpm','ad_totalEcpm']

df = pd.read_csv(path,delimiter = '\t',\
                          parse_dates = ['request_time'],header=None,names = names)

In [4]:
df.head()

,request_id,request_time,location_id,user_id,ad_id,ad_size,ad_bid,ad_pctr,ad_quality_ecpm,ad_totalEcpm
0,84053223,1550332665,240,917194,186369,30,24,3.719,74.38,163.636
1,23482062,1550332515,39,385699,137171,44,312,17.932,358.64,5952.640
2,34020708,1550332384,30,613627,185128,64,22,27.057,541.14,1136.394
3,54435556,1550332542,196,1290252,293117,44,105,12.954,259.08,1619.080
4,9415438,1550330748,58,88563,711505,30,11,68.282,1365.64,2116.742


In [5]:
df['request_datetime'] = pd.to_datetime(df['request_time'],unit='s')

In [6]:
df.head()

,request_id,request_time,location_id,user_id,ad_id,ad_size,ad_bid,ad_pctr,ad_quality_ecpm,ad_totalEcpm,request_datetime
0,84053223,1550332665,240,917194,186369,30,24,3.719,74.38,163.636,2019-02-16 15:57:45
1,23482062,1550332515,39,385699,137171,44,312,17.932,358.64,5952.640,2019-02-16 15:55:15
2,34020708,1550332384,30,613627,185128,64,22,27.057,541.14,1136.394,2019-02-16 15:53:04
3,54435556,1550332542,196,1290252,293117,44,105,12.954,259.08,1619.080,2019-02-16 15:55:42
4,9415438,1550330748,58,88563,711505,30,11,68.282,1365.64,2116.742,2019-02-16 15:25:48


In [7]:
'''
    可以看出每天曝光广告的基本数据，接下来需要构建必要的信息
    创建时间
    广告行业id
    商品类型
    商品ID
    广告账户id
    投放时段
    人群定向
    出价
    
    其中投放时段，人群定向，出价属于动态数据，不同的时间其可能不同
    
    接下来就是要确定曝光的时刻，这些动态的具体数值
    
    怎么确定？
    需要将动态数据和静态数据进行结合
'''
ad_static_path = '../data/testA/ad_static_feature.out'
ad_operation_path = '../data/testA/ad_operation.dat'

static_feature_names = ['ad_id','create_time','ad_acc_id','good_id','good_class','ad_trade_id','ad_size']

ad_static_df = pd.read_csv(ad_static_path,delimiter = '\t',\
                          parse_dates = ['create_time'],header=None,names = static_feature_names,dtype={'ad_id':int,"ad_acc_id": int,\
                          "good_id": str, "good_class": str, "ad_trade_id": str,'ad_size':str}) 

In [8]:
ad_static_df.head()

,ad_id,create_time,ad_acc_id,good_id,good_class,ad_trade_id,ad_size
0,106452,1529958950,22226,16088,13,225,NaN
1,233649,1538221936,25681,7356,13,136,1
2,547531,1550731020,20696,-1,1,186,40
3,707841,1551857857,3968,-1,3,186,40
4,457009,1550439402,23614,7447,13,172,NaN


In [9]:
ad_static_df['create_datetime'] = pd.to_datetime(ad_static_df['create_time'],unit='s')

In [10]:
ad_static_df['create_datetime'] = ad_static_df['create_datetime'].astype(str)

In [11]:
ad_static_df[ad_static_df['ad_id'] == 593323]

,ad_id,create_time,ad_acc_id,good_id,good_class,ad_trade_id,ad_size,create_datetime
561639,593323,1428399835,26889,-1,3,85,64,2015-04-07 09:43:55


In [12]:
operation_names = ['ad_id','create_update_time','op_type','op_set','op_value']

ad_op_df = pd.read_csv(ad_operation_path,delimiter = '\t',header=None,names = operation_names,\
                       dtype={"ad_id": int,'create_update_time':int,"op_type": int,"op_set": int, "op_value": object}) 

In [13]:
ad_op_df.head()

,ad_id,create_update_time,op_type,op_set,op_value
0,593323,0,2,2,90
1,593323,0,2,3,all
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,20190217000000,1,1,0
4,593323,20190218233855,1,2,90


In [14]:
'''
    将create_update_time不为0的时间修改成‘2015-04-07 09:43:55’格式
'''
def convert_time(x):
    x = str(x)
    return x[0:4] + '-' + x[4:6] + '-' + x[6:8] + ' ' + x[8:10] + ':' + x[10:12] + ':' + x[12:14]

ad_op_df.loc[ad_op_df['create_update_time'] != 0,'create_update_time']=ad_op_df['create_update_time'].apply(convert_time)

In [15]:
ad_op_df.head()

,ad_id,create_update_time,op_type,op_set,op_value
0,593323,0,2,2,90
1,593323,0,2,3,all
2,593323,0,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,2019-02-17 00:00:00,1,1,0
4,593323,2019-02-18 23:38:55,1,2,90


In [16]:
'''
    发现很多新建的时间都为0，所以先填充时间
'''
ad_op_df.loc[ad_op_df['create_update_time'] == 0,'create_update_time']=ad_op_df['ad_id'].map(dict(zip(ad_static_df.ad_id,ad_static_df.create_datetime)))

In [17]:
# 终于把时间填补成功
'''
    op_type:操作类型 1：修改 2：新建、
    op_set:修改字段 1-广告状态，2-出价，3-人群定向，4-广告时段设置
    op_value：修改后的值
'''
ad_op_df.head()

,ad_id,create_update_time,op_type,op_set,op_value
0,593323,2015-04-07 09:43:55,2,2,90
1,593323,2015-04-07 09:43:55,2,3,all
2,593323,2015-04-07 09:43:55,2,4,"281474976710655,281474976710655,28147497671065..."
3,593323,2019-02-17 00:00:00,1,1,0
4,593323,2019-02-18 23:38:55,1,2,90


In [18]:
ad_op_df.sort_values(by = ['ad_id','create_update_time'],inplace=True)

In [19]:
ad_op_df.head()

,ad_id,create_update_time,op_type,op_set,op_value
721281,31,2019-03-12 23:22:50,2,2,100
721282,31,2019-03-12 23:22:50,2,3,area:11442
721283,31,2019-03-12 23:22:50,2,4,"281474976694272,281474976694272,28147497669427..."
721284,31,2019-03-13 19:52:07,1,1,0
250461,32,2018-11-30 07:40:17,2,2,83


In [20]:
ad_op_df = ad_op_df.reset_index()
ad_op_df.drop(columns = ['index'],inplace=True)
ad_op_df.head()
# 现在的数据顺序是按照ad_id create_update_time 升序排列

,ad_id,create_update_time,op_type,op_set,op_value
0,31,2019-03-12 23:22:50,2,2,100
1,31,2019-03-12 23:22:50,2,3,area:11442
2,31,2019-03-12 23:22:50,2,4,"281474976694272,281474976694272,28147497669427..."
3,31,2019-03-13 19:52:07,1,1,0
4,32,2018-11-30 07:40:17,2,2,83


In [21]:
# 我现在很好奇一个广告会不会多次失效
ad_op_df[ad_op_df['ad_id'] == 32]
# 还真会，一个广告属性不变，一会生效 一会失效

,ad_id,create_update_time,op_type,op_set,op_value
4,32,2018-11-30 07:40:17,2,2,83
5,32,2018-11-30 07:40:17,2,3,area:7572
6,32,2018-11-30 07:40:17,2,4,"70368475742208,70368475742208,70368475742208,7..."
7,32,2019-02-21 05:31:31,1,2,91
8,32,2019-02-21 22:34:03,1,1,0
9,32,2019-02-22 05:31:39,1,1,1
10,32,2019-02-22 21:52:04,1,1,0
11,32,2019-02-23 05:31:42,1,1,1
12,32,2019-02-23 10:41:13,1,1,0
13,32,2019-02-24 12:11:14,1,1,1


In [22]:
bin(70368475742208)[2:]

'1111111111111111110000000000000000000000000000'

In [23]:
# 将时间转换测试
def convertOneStr2Interval(x):
    x = int(x)
    bin_str = bin(x)[2:]
    bin_len = len(bin_str)
    r_pos = bin_str.rfind('1')
    if bin_len % 2 == 0:
        end_date = str(bin_len//2) + ':00'
    else:
        end_date = str(bin_len//2) + ':30'
    
    interval = bin_len - r_pos - 1
    if interval % 2 == 0:
        begin_date = str(interval//2) + ':00'
    else:
        begin_date = str(interval//2) + ':30'
    return begin_date + '-' + end_date    

In [24]:
def convertStr2Interval(x):
    res_str = ''
    time_list = x.split(',')
    for time in time_list:
        res_str += convertOneStr2Interval(time) + ','
    return res_str[:-1]

In [25]:
convertStr2Interval('70368475742208')

'14:00-23:00'

In [26]:
test_op_df = ad_op_df[ad_op_df['ad_id'].isin([31,32])]

In [37]:
ad_op_df.head()

,ad_id,create_update_time,op_type,op_set,op_value
0,31,2019-03-12 23:22:50,2,2,100
1,31,2019-03-12 23:22:50,2,3,area:11442
2,31,2019-03-12 23:22:50,2,4,"281474976694272,281474976694272,28147497669427..."
3,31,2019-03-13 19:52:07,1,1,0
4,32,2018-11-30 07:40:17,2,2,83


In [28]:
'''
    接下来整合广告的静态数据和动态数据
    新建的dataframe需要的列有
    ad_id
    create_time
    ad_acc_id
    good_id
    good_class
    ad_trade_id
    ad_size
    target_people:目标人群
    deliver_time:投放时段
    ad_bid:出价
    valid_time:广告该设置的有效时间,字符串格式 2019-03-09 13:40:03-2019-03-09 14:40:03
    其中唯一索引是ad_id 和 valid_time
    
    
    在整合动态，静态数据之前，首先需要整合动态数据
    动态数据的列有：
    ad_id
    ad_bid
    deliver_time
    target_people
    valid_time
'''
new_op_df = pd.DataFrame(columns = ['ad_id','ad_bid','deliver_time','target_people','valid_time'])
# 遍历广告的动态流水数据
# ad_id	create_update_time	op_type	op_set	op_value

ad_id = ''
ad_bid = None
deliver_time = ''
target_people = ''
valid_start_time = ''

index = 0
for _,row in test_op_df.iterrows():
    # 如果是新建类型
    if row['op_type'] == 2:
        # 广告出价
        if row['op_set'] ==2:
            ad_bid = int(row['op_value'])
        # 人群定向
        elif row['op_set'] == 3:
            target_people = row['op_value']
        # 投放时段
        elif row['op_set'] == 4:
            deliver_time = convertStr2Interval(row['op_value'])
        # time\id
        valid_start_time = row['create_update_time']
        ad_id = row['ad_id']
        continue
    # 修改类型
    elif row['op_type'] == 1:
        # 是同一个ad_id
        if row['ad_id'] == ad_id:
            # 如果修改的是广告出价
            if row['op_set'] == 2:
                # 需要将之前的数据插入到新的data中
                new_op_df.loc[index,'ad_id'] = ad_id
                new_op_df.loc[index,'ad_bid'] = ad_bid
                new_op_df.loc[index,'deliver_time'] = deliver_time
                new_op_df.loc[index,'target_people'] = target_people
                new_op_df.loc[index,'valid_time'] = valid_start_time + '-' + row['create_update_time']
                
                # 重新设置时间段
                valid_start_time = row['create_update_time']
                ad_bid = int(row['op_value'])
                index += 1
                
            # 如果修改的是投放时段
            if row['op_set'] == 4:
                # 需要将之前的数据插入到新的data中
                new_op_df.loc[index,'ad_id'] = ad_id
                new_op_df.loc[index,'ad_bid'] = ad_bid
                new_op_df.loc[index,'deliver_time'] = deliver_time
                new_op_df.loc[index,'target_people'] = target_people
                new_op_df.loc[index,'valid_time'] = valid_start_time + '-' + row['create_update_time']
                
                # 重新设置时间段
                valid_start_time = row['create_update_time']
                deliver_time = convertStr2Interval(row['op_value'])
                index += 1
                
            # 如果修改的是人群定向
            if row['op_set'] == 3:
                # 需要将之前的数据插入到新的data中
                new_op_df.loc[index,'ad_id'] = ad_id
                new_op_df.loc[index,'ad_bid'] = ad_bid
                new_op_df.loc[index,'deliver_time'] = deliver_time
                new_op_df.loc[index,'target_people'] = target_people
                new_op_df.loc[index,'valid_time'] = valid_start_time + '-' + row['create_update_time']
                
                # 重新设置时间段
                valid_start_time = row['create_update_time']
                target_people = row['op_value']
                index += 1
            
        
            # 如果修改的是广告状态
            if row['op_set'] == 1:
                # 如果设置为无效
                if row['op_value'] == '0':
                    new_op_df.loc[index,'ad_id'] = ad_id
                    new_op_df.loc[index,'ad_bid'] = ad_bid
                    new_op_df.loc[index,'deliver_time'] = deliver_time
                    new_op_df.loc[index,'target_people'] = target_people
                    new_op_df.loc[index,'valid_time'] = valid_start_time + '-' + row['create_update_time']

                    # 重新设置时间段
                    valid_start_time = ''
                    index += 1
                else:
                    valid_start_time = row['create_update_time']

In [29]:
new_op_df

,ad_id,ad_bid,deliver_time,target_people,valid_time
0,31,100,"7:00-24:00,7:00-24:00,7:00-24:00,7:00-24:00,7:...",area:11442,2019-03-12 23:22:50-2019-03-13 19:52:07
1,32,83,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2018-11-30 07:40:17-2019-02-21 05:31:31
2,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-21 05:31:31-2019-02-21 22:34:03
3,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-22 05:31:39-2019-02-22 21:52:04
4,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-23 05:31:42-2019-02-23 10:41:13
5,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-24 12:11:14-2019-02-24 12:25:05
6,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-24 13:11:16-2019-02-24 15:13:05
7,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-25 16:01:16-2019-02-25 16:01:19
8,32,90,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-25 16:01:19-2019-02-25 20:37:03
9,32,90,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-26 05:31:38-2019-02-26 19:34:03


In [30]:
new_op_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 5 columns):
ad_id            42 non-null object
ad_bid           42 non-null object
deliver_time     42 non-null object
target_people    42 non-null object
valid_time       42 non-null object
dtypes: object(5)
memory usage: 3.2+ KB


In [31]:
ad_static_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735911 entries, 0 to 735910
Data columns (total 8 columns):
ad_id              735911 non-null int64
create_time        735911 non-null object
ad_acc_id          735911 non-null int64
good_id            735911 non-null object
good_class         735911 non-null object
ad_trade_id        735911 non-null object
ad_size            509252 non-null object
create_datetime    735911 non-null object
dtypes: int64(2), object(6)
memory usage: 44.9+ MB


In [32]:
test_op_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 72
Data columns (total 5 columns):
ad_id                 73 non-null int64
create_update_time    73 non-null object
op_type               73 non-null int64
op_set                73 non-null int64
op_value              73 non-null object
dtypes: int64(3), object(2)
memory usage: 3.4+ KB


In [33]:
# 看起来处理的好像比较成功，因为都是新建后修改
# 接下来就是处理时间，将字符串转换成范围
ad_static_df['ad_id'] = ad_static_df['ad_id'].astype(object)
# 将这个数据与静态数据合并
new_op_df = new_op_df.merge(ad_static_df,on = 'ad_id',how = 'left')

In [34]:
new_op_df.drop(columns = ['create_time'],inplace=True)

In [35]:
new_op_df

,ad_id,ad_bid,deliver_time,target_people,valid_time,ad_acc_id,good_id,good_class,ad_trade_id,ad_size,create_datetime
0,31,100,"7:00-24:00,7:00-24:00,7:00-24:00,7:00-24:00,7:...",area:11442,2019-03-12 23:22:50-2019-03-13 19:52:07,12577,18683,13,224,40,2019-03-12 23:22:50
1,32,83,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2018-11-30 07:40:17-2019-02-21 05:31:31,18752,32534,13,136,40,2018-11-30 07:40:17
2,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-21 05:31:31-2019-02-21 22:34:03,18752,32534,13,136,40,2018-11-30 07:40:17
3,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-22 05:31:39-2019-02-22 21:52:04,18752,32534,13,136,40,2018-11-30 07:40:17
4,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-23 05:31:42-2019-02-23 10:41:13,18752,32534,13,136,40,2018-11-30 07:40:17
5,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-24 12:11:14-2019-02-24 12:25:05,18752,32534,13,136,40,2018-11-30 07:40:17
6,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-24 13:11:16-2019-02-24 15:13:05,18752,32534,13,136,40,2018-11-30 07:40:17
7,32,91,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-25 16:01:16-2019-02-25 16:01:19,18752,32534,13,136,40,2018-11-30 07:40:17
8,32,90,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-25 16:01:19-2019-02-25 20:37:03,18752,32534,13,136,40,2018-11-30 07:40:17
9,32,90,"14:00-23:00,14:00-23:00,14:00-23:00,14:00-23:0...",area:7572,2019-02-26 05:31:38-2019-02-26 19:34:03,18752,32534,13,136,40,2018-11-30 07:40:17


In [36]:
#new_op_df.to_csv('../data/testA/ad_static_dynamic_merge.csv')